In [ ]:
import os
import numpy as np
import pandas as pd
from plt_rcs import *
import hds
plt.rc(group = 'figure', figsize = (4, 4))

In [ ]:
os.getcwd()

In [ ]:
os.chdir('../../data')

In [ ]:
sorted(os.listdir())

In [ ]:
objs = pd.read_pickle('WhiteWine.pkl')

In [ ]:
globals().update(objs)

In [ ]:
X_tr, X_vl, y_tr, y_vl = X_train, X_valid, y_train, y_valid

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier(n_estimators = 1000, early_stopping_rounds = 50)

In [ ]:
model.fit(X = X_tr, y = y_tr, eval_set = [(X_vl, y_vl)], verbose = False)

In [ ]:
model.score(X = X_tr, y = y_tr)
# 0.9830805134189031
model.score(X = X_vl, y = y_vl)
# 0.8639455782312925

In [ ]:
pd.Series(data = model.feature_importances_, 
          index = model.feature_names_in_) \
  .sort_values(ascending = False)

In [ ]:
hds.plot.feature_importance(model)

In [ ]:
model.get_booster().num_boosted_rounds()
# 109

In [ ]:
model.best_iteration
# 58

In [ ]:
model.best_score
# 0.351749884845728

In [ ]:
y_pred = model.predict(X = X_vl)

In [ ]:
hds.stat.clfmetrics(y_true = y_vl, y_pred = y_pred)

In [ ]:
y_prob = model.predict_proba(X = X_vl)

In [ ]:
hds.plot.roc_curve(y_true = y_vl, y_prob = y_prob)

In [ ]:
hds.plot.pr_curve(y_true = y_vl, y_prob = y_prob)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(k_neighbors = 5, random_state = 0)

In [ ]:
X_bal, y_bal = smote.fit_resample(X = X_tr, y = y_tr)

In [ ]:
model_bal = XGBClassifier(n_estimators = 1000, early_stopping_rounds = 50)

In [ ]:
model_bal.fit(X = X_bal, y = y_bal, eval_set = [(X_vl, y_vl)], verbose = False)

In [ ]:
model_bal.score(X = X_bal, y = y_bal)

In [ ]:
model_bal.score(X = X_vl, y = y_vl)

In [ ]:
model_bal.get_booster().num_boosted_rounds()

In [ ]:
model_bal.best_iteration

In [ ]:
y_pred_bal = model_bal.predict(X = X_vl)

In [ ]:
hds.stat.clfmetrics(y_true = y_vl, y_pred = y_pred_bal)

In [ ]:
y_prob_bal = model_bal.predict_proba(X = X_vl)

In [ ]:
hds.plot.roc_curve(y_true = y_vl, y_prob = y_prob, color = 'red')
hds.plot.roc_curve(y_true = y_vl, y_prob = y_prob_bal, color = 'blue')

In [ ]:
hds.plot.pr_curve(y_true = y_vl, y_prob = y_prob, color = 'red')
hds.plot.pr_curve(y_true = y_vl, y_prob = y_prob_bal, color = 'blue')

In [ ]:
y_vl.value_counts()

In [ ]:
ratio = 1157 / 313

In [ ]:
model_wgt = XGBClassifier(n_estimators = 1000, 
                          early_stopping_rounds = 50, 
                          scale_pos_weight = ratio)

In [ ]:
model_wgt.fit(X = X_tr, y = y_tr, eval_set = [(X_vl, y_vl)], verbose = False)

In [ ]:
model_wgt.score(X = X_tr, y = y_tr)

In [ ]:
model_wgt.score(X = X_vl, y = y_vl)

In [ ]:
y_pred_wgt = model_wgt.predict(X = X_vl)

In [ ]:
hds.stat.clfmetrics(y_true = y_vl, y_pred = y_pred_wgt)

In [ ]:
y_prob_wgt = model_wgt.predict_proba(X = X_vl)

In [ ]:
hds.plot.roc_curve(y_true = y_vl, y_prob = y_prob, color = 'red')
hds.plot.roc_curve(y_true = y_vl, y_prob = y_prob_wgt, color = 'blue')

In [ ]:
hds.plot.pr_curve(y_true = y_vl, y_prob = y_prob, color = 'red')
hds.plot.pr_curve(y_true = y_vl, y_prob = y_prob_wgt, color = 'blue')

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_true = y_vl, y_score = y_prob[:, 1])

In [ ]:
roc_auc_score(y_true = y_vl, y_score = y_prob_wgt[:, 1])